In [ ]:
# copy model from get_data.py
# replace list of cities by just one city.
# remove column 'weathercode'

In [3]:
import pandas as pd
import requests
from datetime import date
from dateutil.relativedelta import relativedelta
from geopy.geocoders import Nominatim

In [4]:
class WeatherForecast:
    def __init__(self, city:str, days:int):
        self.city = city
        self.days = days

    def get_city_lonlan(self):
        '''
        This function receives the name of one city and returns the lat and lon of that city
        in a dictionary
        '''

        # Create a geolocator object
        geolocator = Nominatim(user_agent="my_app")

        #save the coordinates of each city in self.city in a dictionary
        coordinates = {}
        
        # Get the location of the city
        location = geolocator.geocode(self.city)

        #check if the location exists
        if location:
            lat, lon = location.latitude, location.longitude # Extract the latitude and longitude
            coordinates[self.city] = [lat,lon]
        else:
            print(f"Could not retrieve coordinates for {self.city}")

        return coordinates

    def get_weather_forecast(self):

        '''
        This function receives 
            * the name of the city list
            * a number of days of weather forecast we want to work on
            
        This function returns a dataframe with the average of the weather data from these city list during those days of forecast
        '''

        # First we declare the weather parameters. Here we'll be taking all params supported by the API
        weather_params = ['temperature_2m','relativehumidity_2m','dewpoint_2m',
                      'apparent_temperature','pressure_msl','surface_pressure',
                      'precipitation','rain','snowfall','cloudcover',
                      'cloudcover_low','cloudcover_mid','cloudcover_high',
                      'shortwave_radiation','direct_radiation','direct_normal_irradiance',
                      'diffuse_radiation','windspeed_10m','windspeed_120m',
                      'winddirection_10m','winddirection_120m','windgusts_10m',
                      'et0_fao_evapotranspiration','vapor_pressure_deficit',
                      'soil_temperature_0cm','soil_temperature_6cm',
                      'soil_temperature_18cm','soil_temperature_54cm',
                      'soil_moisture_0_1cm','soil_moisture_1_3cm',
                      'soil_moisture_3_9cm','soil_moisture_9_27cm', 'soil_moisture_27_81cm']
        # remove 'weathercode' from weather_params
        
        # Then we compute the dates used to get the weather forecast data
        
        # end_date : today + number of days of forecast (self.days)
        end_date = (date.today() + relativedelta(days=self.days)).strftime('%Y-%m-%d')
        
        #start_date : today at midnight
        start_date = (date.today()).strftime('%Y-%m-%d')

        #call the method to receive the coordinates from the self.city list
        coordinates = self.get_city_lonlan()
        
        #create an empty dataframe
        # cities = []
        
        #create a dataframe with weather params for the selected city and store it
        lat = coordinates[self.city][0]
        lan = coordinates[self.city][1]

    
        # So we make the request to the weather API archive
        weather_forecast_response= requests.get('https://api.open-meteo.com/v1/forecast',
                            params = {'latitude': lat,
                                        'longitude': lan,
                                        'forecast_days' : self.days,
                                        'hourly': weather_params,
                                        'timezone': 'auto'}).json()
        
        weather_forecast_df = pd.DataFrame(weather_forecast_response['hourly'], columns = ['time'] + weather_params)
        weather_forecast_df['time'] = pd.to_datetime(weather_forecast_df['time'], format='%Y-%m-%d')
        weather_forecast_df = weather_forecast_df.set_index('time')

        # Format float to 1 decimal, sum the 3 tables and return the average
        pd.options.display.float_format = "{:,.1f}".format
        
        
        return weather_forecast_df
    
    def rename_columns(self):
        weather_df_old_columns_names = self.get_weather_forecast()
        
        # remove 'weathercode' from this params
        weather_params_history_data = ["time",'temperature_2m','relativehumidity_2m','dewpoint_2m',
                      'apparent_temperature','pressure_msl','surface_pressure',
                      'precipitation','rain','snowfall','cloudcover',
                      'cloudcover_low','cloudcover_mid','cloudcover_high',
                      'shortwave_radiation','direct_radiation','direct_normal_irradiance',
                      'diffuse_radiation','windspeed_10m','windspeed_100m',
                      'winddirection_10m','winddirection_100m','windgusts_10m',
                      'et0_fao_evapotranspiration','vapor_pressure_deficit',
                      'soil_temperature_0_to_7cm','soil_temperature_7_to_28cm',
                      'soil_temperature_28_to_100cm','soil_temperature_100_to_255cm',
                      'soil_moisture_0_to_7cm','soil_moisture_7_to_28cm',
                      'soil_moisture_28_to_100cm','soil_moisture_100_to_255cm']
        
        weather_df_old_columns_names.columns = weather_params_history_data
        return weather_df_old_columns_names


In [5]:
forecast = WeatherForecast(city="Amiens", days=3)

In [6]:
forecast.get_city_lonlan()

{'Amiens': [49.8941708, 2.2956951]}

In [19]:
forecast.get_weather_forecast()

,temperature_2m,relativehumidity_2m,dewpoint_2m,apparent_temperature,pressure_msl,surface_pressure,precipitation,rain,snowfall,cloudcover,...,vapor_pressure_deficit,soil_temperature_0cm,soil_temperature_6cm,soil_temperature_18cm,soil_temperature_54cm,soil_moisture_0_1cm,soil_moisture_1_3cm,soil_moisture_3_9cm,soil_moisture_9_27cm,soil_moisture_27_81cm
time,,,,,,,,,,,,,,,,,,,,,
2023-03-15 00:00:00,3.4,84,1.0,-0.2,"1,015.2","1,010.9",0.0,0.0,0.0,24,...,0.1,2.7,4.8,8.2,8.3,0.3,0.3,0.3,0.3,0.3
2023-03-15 01:00:00,3.5,83,0.8,-0.1,"1,015.3","1,011.0",0.0,0.0,0.0,51,...,0.1,2.9,4.7,7.9,8.3,0.3,0.3,0.3,0.3,0.3
2023-03-15 02:00:00,3.2,86,1.1,-0.3,"1,016.0","1,011.7",0.0,0.0,0.0,49,...,0.1,2.4,4.5,7.7,8.3,0.3,0.3,0.3,0.3,0.3
2023-03-15 03:00:00,3.3,88,1.5,0.1,"1,016.1","1,011.8",0.0,0.0,0.0,100,...,0.1,3.2,4.4,7.4,8.3,0.3,0.3,0.3,0.3,0.3
2023-03-15 04:00:00,3.9,87,1.9,0.6,"1,015.3","1,011.1",1.4,0.0,0.0,80,...,0.1,3.5,4.6,7.2,8.3,0.3,0.3,0.3,0.3,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-17 19:00:00,11.9,86,9.7,10.8,"1,007.6","1,003.5",0.0,0.0,0.0,100,...,0.2,10.4,10.7,9.2,7.7,0.3,0.3,0.3,0.3,0.3
2023-03-17 20:00:00,11.4,89,9.7,10.4,"1,008.1","1,004.0",0.0,0.0,0.0,100,...,0.1,10.1,10.3,9.3,7.8,0.3,0.3,0.3,0.3,0.3
2023-03-17 21:00:00,11.0,91,9.6,9.7,"1,008.4","1,004.3",0.0,0.0,0.0,100,...,0.1,9.6,10.0,9.3,7.8,0.3,0.3,0.3,0.3,0.3


In [13]:
forecast.rename_columns()

,time,temperature_2m,relativehumidity_2m,dewpoint_2m,apparent_temperature,pressure_msl,surface_pressure,precipitation,rain,snowfall,...,et0_fao_evapotranspiration,vapor_pressure_deficit,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_temperature_28_to_100cm,soil_temperature_100_to_255cm,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_moisture_28_to_100cm,soil_moisture_100_to_255cm
time,,,,,,,,,,,,,,,,,,,,,
2023-03-15 00:00:00,3.4,84,1.0,-0.2,"1,015.2","1,010.9",0.0,0.0,0.0,24,...,0.1,2.7,4.8,8.2,8.3,0.3,0.3,0.3,0.3,0.3
2023-03-15 01:00:00,3.5,83,0.8,-0.1,"1,015.3","1,011.0",0.0,0.0,0.0,51,...,0.1,2.9,4.7,7.9,8.3,0.3,0.3,0.3,0.3,0.3
2023-03-15 02:00:00,3.2,86,1.1,-0.3,"1,016.0","1,011.7",0.0,0.0,0.0,49,...,0.1,2.4,4.5,7.7,8.3,0.3,0.3,0.3,0.3,0.3
2023-03-15 03:00:00,3.3,88,1.5,0.1,"1,016.1","1,011.8",0.0,0.0,0.0,100,...,0.1,3.2,4.4,7.4,8.3,0.3,0.3,0.3,0.3,0.3
2023-03-15 04:00:00,3.9,87,1.9,0.6,"1,015.3","1,011.1",1.4,0.0,0.0,80,...,0.1,3.5,4.6,7.2,8.3,0.3,0.3,0.3,0.3,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-17 19:00:00,11.9,86,9.7,10.8,"1,007.6","1,003.5",0.0,0.0,0.0,100,...,0.2,10.4,10.7,9.2,7.7,0.3,0.3,0.3,0.3,0.3
2023-03-17 20:00:00,11.4,89,9.7,10.4,"1,008.1","1,004.0",0.0,0.0,0.0,100,...,0.1,10.1,10.3,9.3,7.8,0.3,0.3,0.3,0.3,0.3
2023-03-17 21:00:00,11.0,91,9.6,9.7,"1,008.4","1,004.3",0.0,0.0,0.0,100,...,0.1,9.6,10.0,9.3,7.8,0.3,0.3,0.3,0.3,0.3


In [10]:
# remove 'weathercode'
weather_params_history_data = ["time",'temperature_2m','relativehumidity_2m','dewpoint_2m',
                      'apparent_temperature','pressure_msl','surface_pressure',
                      'precipitation','rain','snowfall','cloudcover',
                      'cloudcover_low','cloudcover_mid','cloudcover_high',
                      'shortwave_radiation','direct_radiation','direct_normal_irradiance',
                      'diffuse_radiation','windspeed_10m','windspeed_100m',
                      'winddirection_10m','winddirection_100m','windgusts_10m',
                      'et0_fao_evapotranspiration','vapor_pressure_deficit',
                      'soil_temperature_0_to_7cm','soil_temperature_7_to_28cm',
                      'soil_temperature_28_to_100cm','soil_temperature_100_to_255cm',
                      'soil_moisture_0_to_7cm','soil_moisture_7_to_28cm',
                      'soil_moisture_28_to_100cm','soil_moisture_100_to_255cm']

In [11]:
len(weather_params_history_data)

33